In [1]:
# default parameters (controlled by papermill in automatic runs)
target="prod"
run='classic'
sample_orders=None # if None - no voter sampling occurs
search_term="^Hot Shots! .1991." # regular expression for the search term
k=10
min_popularity=20

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")
sys.path.append("../es/")


import random
myseed = 13
random.seed(myseed)

import es.runner.scenario_runner
import es.elections
import es.utils
from es.data.movielens_data import *
from bokeh.io import output_notebook
output_notebook()
%matplotlib inline

pd.set_option('display.max_rows', 40)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pandasql import sqldf
sql = lambda q: sqldf(q, globals())

Loading BokehJS ...

In [3]:
out_folder = f"../out/{target}/movielens-results/{run}"

In [4]:
%%time
idata = es.data.movielens_data.MovielensData(es.data_config.data_movielens_25m)
iframes = es.elections.InstacartDataFrames.from_data(idata, min_popularity=min_popularity)

reading instacart data
reading DataFrame order_products__all from parquet: /Users/gregaw/code/ddr/esearch/es/../in/movielens_25m/order_products__all.parquet
order_products__all contains 15630129 items
WARN: Only products with min_popularity=20 considered (137378 approvals filtered from 15630129 to 15492751)
reading DataFrame orders from parquet: /Users/gregaw/code/ddr/esearch/es/../in/movielens_25m/orders.parquet
orders contains 162414 items
reading DataFrame products from parquet: /Users/gregaw/code/ddr/esearch/es/../in/movielens_25m/products.parquet
products contains 62423 items
reading DataFrame departments from parquet: /Users/gregaw/code/ddr/esearch/es/../in/movielens_25m/departments.parquet
departments contains 1 items
reading DataFrame aisles from parquet: /Users/gregaw/code/ddr/esearch/es/../in/movielens_25m/aisles.parquet
aisles contains 1 items
CPU times: user 1.22 s, sys: 549 ms, total: 1.77 s
Wall time: 1.5 s


In [5]:
PARAMS = es.utils.dotdict(dict(
    exp_name='tmp',
    k=k,
    searchterms=[search_term],
    filtering=dict(sample_orders=sample_orders),
    rules=[
        "HUV_0", "HUV_1", "HUV_2",
        "sa.HUV_1", "sa.HUV_2",
    ], # prefix sa. means 'use simulated annealing'
    anneal_settings= {'tmax': 9900.0, 'tmin': 0.6, 'steps': 50000, 'updates': 10}, # for GAIW
    scorer_metrics=['tfidfto2'],
    scorer_funcs = [es.scorers.AgentSpreadScoringStrategy.from_found_df_sorted_nosearchterm]
))

## Runner

In [6]:
%%time
_runner = es.runner.scenario_runner.ScenarioRunner(iframes, PARAMS)
exp_folder = _runner.run(verbose=1, include_unity=False, output_base_folder=out_folder)

loaded 62423 products and 162414 orders, (15492751 ops)
2022-05-17 12:29:09.812710
dl=13875, avgdl=1091.5, products=13876 metrics_voter_count=162413, real_voter_count=162413
searching for: ^Hot Shots! .1991. : ['Hot Shots! (1991)']
----------
RUN: exp_name=tmp;ut_scorer=AgentSpread(tfidfto2)
filtered iframes, products: 13875, orders:  1765
processing RULE: HUV_0
Using algo/version: g/4
Using optimised HUV_0
greedy score=67057.75782869327 ([9018.888854878263, 8724.343125944022, 7680.180413712414, 7445.931451446565, 6169.659940872101, 6159.13060883231, 5639.841313108697, 5565.962288032733, 5395.015000844719, 5258.804831021447])
processing RULE: HUV_1
Using algo/version: g/4
added member #0: 3258, total committee score 9018.910, added member delta: 9018.910
added member #1: 428, total committee score 15477.377, added member delta: 6458.467
added member #2: 4605, total committee score 20507.602, added member delta: 5030.224
added member #3: 2717, total committee score 25198.271, added memb

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


first state= [2411, 3042, 3777, 4243, 4763, 10682, 10914, 11207, 11227, 13140]
annealing with: {'tmax': 9900.0, 'tmin': 0.6, 'steps': 50000, 'updates': 10}@ 12:29:50


annealing done @ 12:30:15
annealing score=44926.98515686746
processing RULE: sa.HUV_2
Using algo/version: sa/2


 Temperature        Energy    Accept   Improve     Elapsed   Remaining


first state= [414, 970, 1921, 2347, 4093, 7041, 7107, 7348, 7681, 7977]
annealing with: {'tmax': 9900.0, 'tmin': 0.6, 'steps': 50000, 'updates': 10}@ 12:30:16


annealing done @ 12:30:32
annealing score=40220.09025822183
--------------------
RUN: exp_name=tmp;ut_scorer=AgentSpread(tfidfto2), search term: ^Hot Shots! .1991.


rank                                              HUV_0  HUV_0_id  \
0     1     Naked Gun 2 1/2: The Smell of Fear, The (1991)      3869   
1     2                        Hot Shots! Part Deux (1993)       466   
2     3  Naked Gun: From the Files of Police Squad!, Th...      3868   
3     4                                 Top Secret! (1984)      5500   
4     5  Loaded Weapon 1 (National Lampoon's Loaded Wea...      3208   
5     6                              Police Academy (1984)      2378   
6     7                                  Spaceballs (1987)      3033   
7     8                         Last Boy Scout, The (1991)      7007   
8     9                                    Commando (1985)      6664   
9    10          Naked Gun 33 1/3: The Final Insult (1994)       370   

                                               HUV_1  HUV_1_id  \
0     Naked Gun 2 1/2: The Smell of Fear, The (1991)      3869   
1                        Hot Shots! Part Deux (1993)       466   
2                                 Top Secret! (1984)      5500   
3  Loaded Weapon 1 (National Lampoon's Loaded Wea...      3208   
4                             Major League II (1994)     26870   
5                         Last Boy Scout, The (1991)      7007   
6  Yamakasi - Les samouraÃ¯s des temps modernes (...     42385   
7  Naked Gun: From the Files of Police Squad!, Th...      3868   
8                                 Hudson Hawk (1991)      7000   
9                                Silent Movie (1976)      8512   

                                               HUV_2  HUV_2_id  \
0     Naked Gun 2 1/2: The Smell of Fear, The (1991)      3869   
1                        Hot Shots! Part Deux (1993)       466   
2  Loaded Weapon 1 (National Lampoon's Loaded Wea...      3208   
3                             Major League II (1994)     26870   
4                                 Top Secret! (1984)      5500   
5  Yamakasi - Les samouraÃ¯s des temps modernes (...     42385   
6                                 Hudson Hawk (1991)      7000   
7                                Silent Movie (1976)      8512   
8                                     Freaked (1993)     26861   
9                         Last Boy Scout, The (1991)      7007   

                                            sa.HUV_1  sa.HUV_1_id  \
0                        Hot Shots! Part Deux (1993)          466   
1  Loaded Weapon 1 (National Lampoon's Loaded Wea...         3208   
2  Naked Gun: From the Files of Police Squad!, Th...         3868   
3     Naked Gun 2 1/2: The Smell of Fear, The (1991)         3869   
4                                  Blind Fury (1989)         4574   
5                                 Top Secret! (1984)         5500   
6                                    Commando (1985)         6664   
7                                 Hudson Hawk (1991)         7000   
8                             Major League II (1994)        26870   
9  Yamakasi - Les samouraÃ¯s des temps modernes (...        42385   

                                            sa.HUV_2  sa.HUV_2_id  
0                        Hot Shots! Part Deux (1993)          466  
1  Loaded Weapon 1 (National Lampoon's Loaded Wea...         3208  
2     Naked Gun 2 1/2: The Smell of Fear, The (1991)         3869  
3                                 Men at Work (1990)         5438  
4                                 Top Secret! (1984)         5500  
5                                 Hudson Hawk (1991)         7000  
6                                Silent Movie (1976)         8512  
7                                     Freaked (1993)        26861  
8                             Major League II (1994)        26870  
9  Yamakasi - Les samouraÃ¯s des temps modernes (...        42385

product_id  order_count_found  \
4636        5541               1765   
3258        3869                684   
428          466                926   
3257        3868                978   
4605        5500                463   
2717        3208                185   
1988        2378                454   
2564        3033                845   
5712        7007                233   
5449        6664                262   

                                           product_name  order_count    tf  \
4636                                  Hot Shots! (1991)       1765.0  1765   
3258     Naked Gun 2 1/2: The Smell of Fear, The (1991)       2454.0   684   
428                         Hot Shots! Part Deux (1993)       4238.0   926   
3257  Naked Gun: From the Files of Police Squad!, Th...       5698.0   978   
4605                                 Top Secret! (1984)       1777.0   463   
2717  Loaded Weapon 1 (National Lampoon's Loaded Wea...        543.0   185   
1988                              Police Academy (1984)       2343.0   454   
2564                                  Spaceballs (1987)       7422.0   845   
5712                         Last Boy Scout, The (1991)        934.0   233   
5449                                    Commando (1985)       1189.0   262   

           idf     idfto2        tfidf      tfidfto2       bm25   ratioIn  
4636  4.521992  16.149101  7981.315498  28503.163913  13.390860  1.000000  
3258  4.192423  13.185510  2867.617419   9018.888855  12.166708  0.278729  
428   3.646051   9.421537  3376.243239   8724.343126  10.672141  0.218499  
3257  3.350027   7.852945  3276.326638   7680.180414   9.818363  0.171639  
4605  4.515216  16.081925  2090.544973   7445.931451  12.902439  0.260551  
2717  5.700788  33.349513  1054.645860   6169.659941  15.205291  0.340700  
1988  4.238710  13.566367  1924.374483   6159.130609  12.100920  0.193769  
2564  3.085694   6.674368  2607.411355   5639.841313   9.010946  0.113851  
5712  5.158421  23.888250  1201.912166   5565.962288  14.080433  0.249465  
5449  4.917030  20.591660  1288.261822   5395.015001  13.556781  0.220353

saved election results to ../out/prod/results-movielens/tmp/exp=tmp,search=^Hot Shots! .1991.,win=10,.csv
saved tfidf results to ../out/prod/results-movielens/tmp/exp=tmp,search=^Hot Shots! .1991.,win=10,_TFIDF.tsv
Results saved to '../out/prod/results-movielens/tmp'
CPU times: user 2min 20s, sys: 3.31 s, total: 2min 24s
Wall time: 1min 25s


In [7]:
import os
os.makedirs(out_folder, exist_ok=True)

df = pd.read_csv(f"{exp_folder}/exp=tmp,search=^Hot Shots! .1991.,win=10,.csv")
table_tex = ""
for rank in range(1,k+1):
    row_items = [f"{rank}."]
    for algo, rulex in [
        ('g', 'HUV_0'),
        ('sa', 'HUV_1'),
        ('sa', 'HUV_2'),
        ('g', 'HUV_1'),
        ('g', 'HUV_2'),
    ]:
        movie = df.query(f"rank == {rank} and algo=='{algo}' and rulex=='{rulex}'").product_name.tolist()[0]
        for was, shouldbe in [
            ("Naked Gun 2 1/2: The Smell of Fear, The (1991)", "The Naked Gun 2 1/2 (1991)"),
            ("Naked Gun: From the Files of Police Squad!, The (1988)", "The Naked Gun (1988)"),
            ("Naked Gun 33 1/3: The Final Insult (1994)", "The Naked Gun 33 1/3 (1994)"),
            ("Loaded Weapon 1 (National Lampoon's Loaded Weapon 1) (1993)", "The Loaded Weapon 1 (1993)"),
            ("Yamakasi - Les samouraÃ¯s des temps modernes (2001)", "Yamakasi (2001)")
        ]:
            movie = movie.replace(was, shouldbe)
        row_items.append(movie)
    table_tex += " & ".join(row_items) + " \\\\ \n"

print(table_tex)
filepath = f"{out_folder}/hotshots-results.tex"
with open(filepath, 'w') as f:
    f.write(table_tex)
    print(f"saved hotshots results to {filepath}")

1. & The Naked Gun 2 1/2 (1991) & Hot Shots! Part Deux (1993) & Hot Shots! Part Deux (1993) & The Naked Gun 2 1/2 (1991) & The Naked Gun 2 1/2 (1991) \\ 
2. & Hot Shots! Part Deux (1993) & The Loaded Weapon 1 (1993) & The Loaded Weapon 1 (1993) & Hot Shots! Part Deux (1993) & Hot Shots! Part Deux (1993) \\ 
3. & The Naked Gun (1988) & The Naked Gun (1988) & The Naked Gun 2 1/2 (1991) & Top Secret! (1984) & The Loaded Weapon 1 (1993) \\ 
4. & Top Secret! (1984) & The Naked Gun 2 1/2 (1991) & Men at Work (1990) & The Loaded Weapon 1 (1993) & Major League II (1994) \\ 
5. & The Loaded Weapon 1 (1993) & Blind Fury (1989) & Top Secret! (1984) & Major League II (1994) & Top Secret! (1984) \\ 
6. & Police Academy (1984) & Top Secret! (1984) & Hudson Hawk (1991) & Last Boy Scout, The (1991) & Yamakasi (2001) \\ 
7. & Spaceballs (1987) & Commando (1985) & Silent Movie (1976) & Yamakasi (2001) & Hudson Hawk (1991) \\ 
8. & Last Boy Scout, The (1991) & Hudson Hawk (1991) & Freaked (1993) & The Na